## Example Coregistration using Geometry + Ancillary-based Refinement

Location: Hawaii    
Data: ALOS     
Link: https://nci-data-training.readthedocs.io/en/latest/_notebook/eo/Python_GDAL_NetCDF.html

#### Prepare for `stripmapApp.py`

Link: https://github.com/parosen/Geo-SInC/blob/main/UNAVCO2021/1.4_Stripmap_Data_Processing_Interferometry/stripmapApp.ipynb

+ Search and download ALOS data from ASF

+ DEM

```bash
cd ~/data/geolocation/HawaiiAlosDT598
mkdir -p DEM; cd DEM
dem.py -a stitch -b 18 21 -156 -154 -r -s 1 -c -f
```

+ reference/secondary.xml
+ stripmapApp.xml

#### Run `stripmapApp.py`

1. Run `stripmapApp.py stripmapApp.xml --end=misregistration`

   - misreg: az = 0.5257771849755031
   - misreg: rg = 0.989622119090436


2. Coregister using geometry without refinement. 

   - Set values to zero in `misreg/misreg_rg.xml` and `misreg/misreg_az.xml` file.
   - Run `stripmapApp.py stripmapApp.xml --start=refined_resample`
   - Rename resulting interferograms for backup


3. Coregister using geometry with ancillary-based refinement

   - Set values to ancillary-based prediction (0.88043165) in `misreg/misreg_rg.xml` file.
   - Run `stripmapApp.py stripmapApp.xml --start=refined_resample`
   - Rename resulting interferograms for backup


In [1]:
%matplotlib inline
import os
import numpy as np
import datetime as dt
from scipy import interpolate
from osgeo import gdal
from matplotlib import pyplot as plt
from mintpy.utils import readfile, utils as ut
from mintpy import tropo_pyaps3 as tropo
import pyaps3 as pa
from tools.simulation import iono
import pysolid

proj_dir = os.path.expanduser('~/data/geolocation/HawaiiAlosDT598/')
os.chdir(proj_dir)
print('Go to directory:', proj_dir)

# ALOS info
c = 299792458
f0 = 1269999750.0604727
rg_pixel_size = 9.3685143125

Go to directory: /Users/yunjunz/data/geolocation/HawaiiAlosDT598/


In [2]:
# geometry: lat/lon/inc_angle
lat_file = os.path.join(proj_dir, 'geometry', 'lat.rdr')
lon_file = os.path.join(proj_dir, 'geometry', 'lon.rdr')
los_file = os.path.join(proj_dir, 'geometry', 'los.rdr')
hgt_file = os.path.join(proj_dir, 'geometry', 'z.rdr')
lat = np.nanmedian(readfile.read(lat_file)[0])
lon = np.nanmedian(readfile.read(lon_file)[0])
hgt = np.nanmedian(readfile.read(hgt_file)[0])
inc_angle = np.nanmedian(readfile.read(los_file, datasetName='incidenceAngle')[0])
az_angle  = np.nanmedian(readfile.read(los_file, datasetName='azimuthAngle')[0])

In [3]:
## Ionosphere

# time:
tmid = dt.datetime.fromisoformat('2011-03-06T20:32:43.389354')
tmin = tmid.hour * 60 + tmid.minute + tmid.second / 60

# geometry: lat/lon/inc_angle at the ionosphere shell
iono_height = 450e3
iono_inc_angle = iono.incidence_angle_ground2iono(inc_angle, iono_height=iono_height)
iono_lat, iono_lon = iono.lalo_ground2iono(lat, lon, inc_angle=inc_angle, az_angle=az_angle)

# geometry: JHR GIM grid
lats = np.arange(-90, 90, step=1) + 0.5
lons = np.arange(-180, 180, step=1) + 0.5
mins = np.arange(0, 24*60, step=15) + 7.5
time_ind = np.argmin(np.abs(mins - tmin))

# calculation/interpolation
tec_vals = []
fnames = [os.path.join(proj_dir, 'GIM', x) for x in ['jpli0190.11i.nc', 'jpli0650.11i.nc']]
for fname in fnames:
    ds = gdal.Open(fname, gdal.GA_ReadOnly)
    tec_map = gdal.Open(ds.GetSubDatasets()[0][0]).ReadAsArray()
    tec_val = interpolate.interp2d(lons, lats, tec_map[time_ind, :, :], kind='linear')(iono_lon, iono_lat)[0]
    tec_vals.append(tec_val)
tec_diff = np.diff(tec_vals)[0]
iono_off = iono.vtec2range_delay(tec_diff, iono_inc_angle, f0) / rg_pixel_size
print('ionospheric range offset: {} pixel'.format(iono_off))

ionospheric range offset: 0.8498202416308941 pixel


In [27]:
## Solid earth tide
dt_obj0 = dt.datetime(2011, 1, 9, 20, 32, 44)
dt_obj1 = dt.datetime(2011, 3, 6, 20, 32, 44) + dt.timedelta(seconds=15)
step_sec = 15

# calculate
(dt_out,
 tide_e,
 tide_n,
 tide_u) = pysolid.calc_solid_earth_tides_point(lat, lon, dt_obj0, dt_obj1, step_sec=step_sec, verbose=False)
tide_e = tide_e[-1] - tide_e[0]
tide_n = tide_n[-1] - tide_n[0]
tide_u = tide_u[-1] - tide_u[0]
tide_los = ut.enu2los(tide_e, tide_n, tide_u, inc_angle=inc_angle, az_angle=az_angle)
tide_off = np.abs(tide_los) / rg_pixel_size
print('SET range offset: {} pixel'.format(tide_off))

PYSOLID: calculate solid Earth tides in east/north/up direction
PYSOLID: lot/lon: 19.50479061273407/-154.98311663649145 degree
PYSOLID: start UTC: 2011-01-09T20:32:44
PYSOLID: end   UTC: 2011-03-06T20:32:59
PYSOLID: time step: 15 seconds
SET range offset: 0.014038569474196525 pixel


In [28]:
## Troposphere
date_list = ['20110109', '20110306']
snwe = [15, 25, -160, -150]
grib_dir = os.path.join(os.path.expandvars('$WEATHER_DIR'), 'ERA5')
grib_files = tropo.get_grib_filenames(date_list=date_list, hour=21, model='ERA5', grib_dir=grib_dir, snwe=snwe)
grib_files = tropo.dload_grib_files(grib_files, tropo_model='ERA5', snwe=snwe)
tropo_los = np.zeros(2, dtype=np.float32)
for i, grib_file in enumerate(grib_files):
    tropo_los[i] = tropo.get_delay(grib_file, tropo_model='ERA5', delay_type='comb',
                                   dem=np.array(hgt, dtype=np.float32).reshape(1,1),
                                   inc=np.array(inc_angle, dtype=np.float32).reshape(1,1),
                                   lat=np.array(lat, dtype=np.float32).reshape(1,1),
                                   lon=np.array(lon, dtype=np.float32).reshape(1,1))
tropo_off = np.abs(np.diff(tropo_los)) / rg_pixel_size
print('tropospheric range offset: {} pixel'.format(tropo_off))


------------------------------------------------------------------------------
downloading weather model data using PyAPS ...
common file size: 386280 bytes
number of grib files existed    : 2
number of grib files to download: 0
------------------------------------------------------------------------------

tropospheric range offset: [0.0165731] pixel


In [29]:
print('total range offset: {} pixel'.format(iono_off + tide_off + tropo_off))

total range offset: [0.88043165] pixel
